In [1]:
color_temp_to_rgb = {}

with open("temp-to-rgb.txt") as f:
  for line in f:
    if line[0] == "#":
      continue
    
    line_respaced = ""
    for i in range(1, len(line)):
      if not (i > 0 and line[i] == ' ' and line[i-1] == ' '):
        line_respaced += line[i]
    
    line_respaced_split = line_respaced.split(" ")
    color_temp = int(line_respaced_split[0])
    r, g, b = [float(x) * 255 for x in line_respaced_split[6:9]]
    
    # 10deg figures overwrite the 2deg ones
    # so the actual RGB values are for CIE 1964 10 degree CMFs, whatever that means
    color_temp_to_rgb[color_temp] = (r, g, b)

In [64]:
from PIL import Image

def convert_img_to_temp(img, temp):
  r, g, b = color_temp_to_rgb[temp]
  matrix = (r / 255., 0., 0., 0.,
            0., g / 255., 0., 0.,
            0., 0., b / 255., 0.,)
  return img.convert("RGB", matrix)

img_filename = "image_0223.png"
img = Image.open(f"test/{img_filename}").convert("RGB")

for temp in range(4000, 10000, 1000):
  convert_img_to_temp(img, temp).save(f"{img_filename[:-4]}-{temp}.png")

In [66]:
temp = 5500
convert_img_to_temp(img, temp).save(f"{img_filename[:-4]}-{temp}.png")

In [2]:
from tqdm import tqdm
from glob import glob
import os
from PIL import Image
from multiprocessing import Pool

temp = 6000

output_dirname = f"../RT-DETR/dataset/test-temp={temp}"
os.makedirs(output_dirname)

def load_and_convert_img(img_path):
  img = Image.open(img_path).convert("RGB")
  output_path = f"{output_dirname}/{img_path.split('/')[-1][:-4]}-{temp}.png"
  convert_img_to_temp(img, temp).save(output_path)
  return output_path

with Pool(16) as pool:
  output_paths = pool.map(load_and_convert_img, glob("../RT-DETR/dataset/test/*.png"))

# for img_path in tqdm(glob("test/*.png")):
#   img = Image.open(img_path).convert("RGB")
#   convert_img_to_temp(img, temp).save(f"{output_dirname}/{img_filename[:-4]}-{temp}.png")

FileExistsError: [Errno 17] File exists: '../RT-DETR/dataset/test-temp=6000'